---
comments: false
layout: post
title: Sprite Animation
description: Testing2
type: hacks
courses: { compsci: {week: 0} }
image: /images/sprite1.png
sprite: /images/sprite1.png
---

In [ ]:
%%html
<!-- Liquid code, run by Jekyll, used to define location of asset(s) -->
{% assign backgroundFile = site.baseurl | append: page.image %}
{% assign spriteImage = site.baseurl | append: page.sprite %}

<style>
    #controls {
        position: relative;
        z-index: 2; /* Ensure the controls are on top of the rob canvas */
    }

    /* Style the rob canvas to be the same size as the viewport */
    #robCanvas {
        position: absolute;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        z-index: 0; /* Place it below the background */
    }
</style>

<!-- Prepare DOM elements -->
<!-- Wrap both the rob canvas and controls in a container div -->
<div id="canvasContainer">
    <div id="controls"> <!-- Controls -->
        <button id="toggleCanvasEffect">Invert</button>

        <input type="radio" name="animation" id="idle">
        <label for="idle">Idle</label>
        <input type="radio" name="animation" id="barking">
        <label for="barking">Barking</label>
        <input type="radio" name="animation" id="walking" checked>
        <label for="walking">Walking</label>
    </div>
    <canvas id="backgroundID">
        <img id="backgroundImage" src="{{backgroundFile}}">
    </canvas>
</div>

<script>
/* Background part of Game
 * scrolling 
*/
// Prepare Background Image
const backgroundImg = new Image();
backgroundImg.src = '{{backgroundFile}}';  // Jekyll/Liquid puts filename here

// Prepare Sprite Image
const robImg = new Image();
robImg.src = '{{spriteImage}}';

// Prepare Canvas
const canvas = document.getElementById("backgroundID");
const ctx = canvas.getContext('2d');

// rob animation part
const robCanvas = document.createElement("canvas");
const robCtx = robCanvas.getContext("2d");

// Prepare Window extents related to viewport
const maxWidth = window.innerWidth;
const maxHeight = window.innerHeight;

backgroundImg.onload = function () {
    // Setup background constants from background image
    const WIDTH = backgroundImg.width;  // Image() width (meta data)
    const HEIGHT = backgroundImg.height; // Image() height
    const ASPECT_RATIO = WIDTH / HEIGHT;
    const ADJUST = 1.42 // visual layer adjust, use "1"" for a perfect loop 

    // Set Dimensions to match the image width
    const canvasWidth = maxWidth;
    const canvasHeight = canvasWidth / ASPECT_RATIO;  // height is oriented by width
    const canvasLeft = 0; // Start image from the left edge horizontally
    const canvasTop = (maxHeight - canvasHeight) / 2;  // center image vertically

    // Set Style properties for the background canvas
    canvas.width = WIDTH / ADJUST;
    canvas.height = HEIGHT / ADJUST;
    canvas.style.width = `${canvasWidth}px`;
    canvas.style.height = `${canvasHeight}px`;
    canvas.style.position = 'absolute';
    canvas.style.left = `${canvasLeft}px`;
    canvas.style.top = `${canvasTop}px`;

    // Game speed is a common game variable
    var gameSpeed = 2;

    // Layer is set up to support Parallax, multiple layers
    class Layer {
        constructor(image, speedRatio) {
            this.x = 0;
            this.y = 0;
            this.width = WIDTH;
            this.height = HEIGHT;
            this.image = image;
            this.speedRatio = speedRatio;
            this.speed = gameSpeed * this.speedRatio;
            this.frame = 0;
        }
        update() {
            this.x = (this.x - this.speed) % this.width;
        }
        draw() {
            ctx.drawImage(this.image, this.x, this.y);
            ctx.drawImage(this.image, this.x + this.width, this.y);
        }
    }

    // Setup rob sprite constraints
    const SPRITE_WIDTH = 61;  // matches sprite pixel width
    const SPRITE_HEIGHT = 65; // matches sprite pixel height
    const SPRITE_FRAMES = 5;  // matches number of frames per sprite row; this code assumes each row is the same
    const SPRITE_SCALE = 1;  // controls the size of the sprite on the canvas

    class rob extends Layer {
        constructor(image, speedRatio) {
            super(image, speedRatio);
            this.minFrame = 0;
            this.maxFrame = SPRITE_FRAMES;
            this.frameX = 0;
            this.frameY = 2;  // walking as default
            this.robX = canvasWidth; // Initialize the rob's x position to the right edge of the canvas
        }
    
        update() {
            if (this.frameY == 2) {
                this.robX -= this.speed;  // Move the rob to the left
                // Check if the rob has moved off the left edge of the canvas
                if (this.robX < -robCanvas.width) {
                    this.robX = canvasWidth; // Reset the rob's x position to the right edge
                }
            }
            // Update frameX of the object
            if (this.frameX < this.maxFrame) {
                this.frameX++;
            } else {
                this.frameX = 0;
            }
        }
    
        // Draw rob object
        draw() {
            // Set fixed dimensions and position for the robCanvas
            robCanvas.width = SPRITE_WIDTH * SPRITE_SCALE;
            robCanvas.height = SPRITE_HEIGHT * SPRITE_SCALE;
            robCanvas.style.width = `${robCanvas.width}px`;
            robCanvas.style.height = `${robCanvas.height}px`;
            robCanvas.style.position = 'absolute';
            robCanvas.style.left = `${this.robX}px`; // Set the rob's left position based on its x-coordinate
            robCanvas.style.top = `${canvasHeight}px`;
    
            robCtx.drawImage(
                this.image,
                this.frameX * SPRITE_WIDTH,
                this.frameY * SPRITE_HEIGHT,
                SPRITE_WIDTH,
                SPRITE_HEIGHT,
                0,
                0,
                robCanvas.width,
                robCanvas.height
            );
        }
    }
    

    // Background object
    var backgroundObj = new Layer(backgroundImg, 0.2);
    var robObj = new rob(robImg, 0.5);

    // Append the rob canvas to the body
    document.body.appendChild(robCanvas);

    // Animation loop
    function animation() {
        backgroundObj.update();
        backgroundObj.draw();

        robObj.update();
        robObj.draw();

        requestAnimationFrame(animation);  // cycle animation, recursion
    }

    // Start animation process
    animation();

    /* Control "rob action" 
     * changes y value, the row in sprite
    */
    // update frameY of rob object, action from idle, bark, walk radio control
    const controls = document.getElementById('controls');
    controls.addEventListener('click', function (event) {
        if (event.target.tagName === 'INPUT') {
            const selectedAnimation = event.target.id;
            switch (selectedAnimation) {
                case 'idle':
                    robObj.frameY = 0;
                    break;
                case 'barking':
                    robObj.frameY = 1;
                    break;
                case 'walking':
                    robObj.frameY = 2;
                    break;
                default:
                    break;
            }
        }
    });
};
</script>

In [2]:
%%html
// SPRITE

{% assign spriteImage = site.baseurl | append: page.image %}

<body>
    <div id="controls"> <!--basic radio buttons which can be used to check whether each individual animaiton works -->
        <button id="toggleCanvasEffect">Invert</button>
        <input type="radio" name="animation" id="walkRight" checked>
        <label for="walkRight">Walk Right</label>
        <input type="radio" name="animation" id="walkLeft">
        <label for="walkLeft">Walk Left</label>
        <input type="radio" name="animation" id="jumpRight">
        <label for="jumpRight">Jump Right</label>
    </div>
    <div>
        <canvas id="spriteContainer"> <!-- Within the base div is a canvas. An HTML canvas is used only for graphics. It allows the user to access some basic functions related to the image created on the canvas (including animation) -->
            <img id="RobotSprite" src="{{spriteImage}}">  <!-- Change sprite here -->
        </canvas>
    </div>
</body>

<script>
    const canvas = document.getElementById('spriteContainer');
    const ctx = canvas.getContext('2d');
    /* Toggle "canvas filter propery" 
    * look in _sass/minima/dark-mode.scss
    */
    var isFilterEnabled = true;
    const defaultFilter = getComputedStyle(document.documentElement).getPropertyValue('--default-canvas-filter');
    toggleCanvasEffect.addEventListener("click", function () {
        if (isFilterEnabled) {
            canvas.style.filter = "none";  // remove filter
        } else {
            canvas.style.filter = defaultFilter; // Apply the default filter value
        }

        isFilterEnabled = !isFilterEnabled;  // switch boolean value
    });
    // start on page load
    window.addEventListener('load', function () {
        const SPRITE_WIDTH = 40;  // matches sprite pixel width
        const SPRITE_HEIGHT = 40; // matches sprite pixel height
        const FRAME_LIMIT = 15;  // matches number of frames per sprite row, this code assume each row is same

        const SCALE_FACTOR = 2;  // control size of sprite on canvas
        canvas.width = SPRITE_WIDTH * SCALE_FACTOR;
        canvas.height = SPRITE_HEIGHT * SCALE_FACTOR;

        // a class to store the differences in the animations to make it clear what the animation changes are for
        class AnimationType{
            constructor(initFrameX = 0, maxFrame = FRAME_LIMIT, animationDelay = 150){
                this.maxFrame = maxFrame;
                this.initFrameX = initFrameX;
                this.animationDelay = animationDelay;
            }
        }

        class Robot {
            constructor() {
                this.image = document.getElementById("RobotSprite");
                this.x = 0;
                this.y = 0;
                this.minFrame = 0;
                this.animationType = new AnimationType();
                this.frameX = 0;
                this.frameY = 0;
            }

            // draw Robot object
            draw(context) {
                context.drawImage(
                    this.image,
                    this.frameX * SPRITE_WIDTH,
                    this.frameY * SPRITE_HEIGHT,
                    SPRITE_WIDTH,
                    SPRITE_HEIGHT,
                    this.x,
                    this.y,
                    canvas.width,
                    canvas.height
                );
            }

            // update frameX of object
            update() {
                // this sprite sheet uses a variable number of frames for different animations
                if (this.frameX < this.animationType.maxFrame) {
                    this.frameX++;
                } else {
                    // the start frame of the animations changes in this sprite sheet
                    this.frameX = this.animationType.initFrameX;
                }
            }
        }

        // Robot object
        const Robot = new Robot();

        // Make the animations that deviate from the default use a class 
        // to make it more readible what the changes are doing
        const jumpRightAnimation = new AnimationType(undefined, 7, undefined); // frames go from 0 to 7 but everything else is the same
        const jumpLeftAnimation = new AnimationType(8, undefined, undefined); // frames go from 8 to 14 but everything else is the same

        // update frameY of Robot object, action from radio controls
        const controls = document.getElementById('controls');
        controls.addEventListener('click', function (event) {
            if (event.target.tagName === 'INPUT') {
                const selectedAnimation = event.target.id;
                switch (selectedAnimation) {
                    case 'walkRight':
                        Robot.frameY = 0;
                        Robot.animationType = new AnimationType();
                        break;
                    case 'walkLeft':
                        Robot.frameY = 1;
                        Robot.animationType = new AnimationType();
                        break;
                    case 'jumpRight':
                        Robot.frameY = 2;
                        Robot.animationType = jumpRightAnimation;
                        break;
                    case 'jumpLeft':
                        // this animation is on the same line as jumpRight
                        Robot.frameY = 2;
                        Robot.animationType = jumpLeftAnimation;
                    default:
                        break;
                }
            }
        });

        // Animation recursive control function
        function animate() {
            // Clears the canvas to remove the previous frame.
            ctx.clearRect(0, 0, canvas.width, canvas.height);

            // Draws the current frame of the sprite.
            Robot.draw(ctx);

            // Updates the `frameX` property to prepare for the next frame in the sprite sheet.
            Robot.update();

            // Uses `requestAnimationFrame` to synchronize the animation loop with the display's refresh rate,
            // ensuring smooth visuals.
            requestAnimationFrame(function() {
                setTimeout(animate, Robot.animationType.animationDelay); // Adjust the delay (in milliseconds) to control the frame rate.
            });
        }

        // run 1st animate
        animate();
    });
</script>



In [2]:
%%html
{% assign spriteImage = site.baseurl | append: page.image %}
SPRITE
<body>
    <div id="controls"> <!--basic radio buttons which can be used to check whether each individual animaiton works -->
        <button id="toggleCanvasEffect">Invert</button>
        <input type="radio" name="animation" id="walkRight" checked>
        <label for="walkRight">Walk Right</label>
        <input type="radio" name="animation" id="walkLeft">
        <label for="walkLeft">Walk Left</label>
        <input type="radio" name="animation" id="jumpRight">
        <label for="jumpRight">Jump Right</label>
    </div>
    <div>
        <canvas id="spriteContainer"> <!-- Within the base div is a canvas. An HTML canvas is used only for graphics. It allows the user to access some basic functions related to the image created on the canvas (including animation) -->
            <img id="robotSprite" src="{{spriteImage}}">  <!-- Change sprite here -->
        </canvas>
    </div>
</body>

<script>
    const canvas = document.getElementById('spriteContainer');
    const ctx = canvas.getContext('2d');
    /* Toggle "canvas filter propery" 
    * look in _sass/minima/dark-mode.scss
    */
    var isFilterEnabled = true;
    const defaultFilter = getComputedStyle(document.documentElement).getPropertyValue('--default-canvas-filter');
    toggleCanvasEffect.addEventListener("click", function () {
        if (isFilterEnabled) {
            canvas.style.filter = "none";  // remove filter
        } else {
            canvas.style.filter = defaultFilter; // Apply the default filter value
        }

        isFilterEnabled = !isFilterEnabled;  // switch boolean value
    });
    // start on page load
    window.addEventListener('load', function () {
        const SPRITE_WIDTH = 83;  // matches sprite pixel width
        const SPRITE_HEIGHT = 49; // matches sprite pixel height
        const FRAME_LIMIT = 7;  // matches number of frames per sprite row, this code assume each row is same

        const SCALE_FACTOR = 2;  // control size of sprite on canvas
        canvas.width = SPRITE_WIDTH * SCALE_FACTOR;
        canvas.height = SPRITE_HEIGHT * SCALE_FACTOR;

        // a class to store the differences in the animations to make it clear what the animation changes are for
        class AnimationType{
            constructor(initFrameX = 0, maxFrame = FRAME_LIMIT, animationDelay = 75){
                this.maxFrame = maxFrame;
                this.initFrameX = initFrameX;
                this.animationDelay = animationDelay;
            }
        }

        class Robot {
            constructor() {
                this.image = document.getElementById("robotSprite");
                this.x = 0;
                this.y = 0;
                this.minFrame = 0;
                this.animationType = new AnimationType();
                this.frameX = 0;
                this.frameY = 0;
            }

            // draw Robot object
            draw(context) {
                context.drawImage(
                    this.image,
                    this.frameX * SPRITE_WIDTH,
                    this.frameY * SPRITE_HEIGHT,
                    SPRITE_WIDTH,
                    SPRITE_HEIGHT,
                    this.x,
                    this.y,
                    canvas.width,
                    canvas.height
                );
            }

            // update frameX of object
            update() {
                // this sprite sheet uses a variable number of frames for different animations
                if (this.frameX < this.animationType.maxFrame) {
                    this.frameX++;
                } else {
                    // the start frame of the animations changes in this sprite sheet
                    this.frameX = this.animationType.initFrameX;
                }
            }
        }

        // Robot object
        const robot = new Robot();

        // Make the animations that deviate from the default use a class 
        // to make it more readible what the changes are doing
        const jumpRightAnimation = new AnimationType(undefined, 7, undefined); // frames go from 0 to 7 but everything else is the same
        const jumpLeftAnimation = new AnimationType(8, undefined, undefined); // frames go from 8 to 14 but everything else is the same

        // update frameY of robot object, action from radio controls
        const controls = document.getElementById('controls');
        controls.addEventListener('click', function (event) {
            if (event.target.tagName === 'INPUT') {
                const selectedAnimation = event.target.id;
                switch (selectedAnimation) {
                    case 'walkRight':
                        robot.frameY = 0;
                        robot.animationType = new AnimationType();
                        break;
                    case 'walkLeft':
                        robot.frameY = 1;
                        robot.animationType = new AnimationType();
                        break;
                    case 'jumpRight':
                        robot.frameY = 2;
                        robot.animationType = jumpRightAnimation;
                        break;
                    case 'jumpLeft':
                        // this animation is on the same line as jumpRight
                        robot.frameY = 2;
                        robot.animationType = jumpLeftAnimation;
                    default:
                        break;
                }
            }
        });

        // Animation recursive control function
        function animate() {
            // Clears the canvas to remove the previous frame.
            ctx.clearRect(0, 0, canvas.width, canvas.height);

            // Draws the current frame of the sprite.
            robot.draw(ctx);

            // Updates the `frameX` property to prepare for the next frame in the sprite sheet.
            robot.update();

            // Uses `requestAnimationFrame` to synchronize the animation loop with the display's refresh rate,
            // ensuring smooth visuals.
            requestAnimationFrame(function() {
                setTimeout(animate, robot.animationType.animationDelay); // Adjust the delay (in milliseconds) to control the frame rate.
            });
        }

        // run 1st animate
        animate();
    });
</script>